In [3]:
import numpy as np

def expon(mean):
    return np.random.exponential(scale=mean)

In [46]:
MEAN_BREAK = 8.0
MEAN_REPAIR = 2.0

NUM_MACHINES = 5
NUM_TECHNICIANS = 2

def initialize():
    global SIM_TIME, Q_TIME_ARRIVAL, Q_MACHINE, TIME_NEXT_EVENT, NUM_EVENTS, TIME_LAST_EVENT, NUM_IN_Q, MACHINE_STATUS, TECHNICIAN_STATUS, NUM_CUSTS_DELAYED, TOTAL_OF_DELAYS, AREA_NUM_IN_Q, TOTAL_TECH_COST, TOTAL_BREAKDOWN_COST
    # sim clock
    SIM_TIME = 0.0

    # queue
    Q_TIME_ARRIVAL = []
    Q_MACHINE = [] # store machine indices

    # event list
    TIME_NEXT_EVENT = [SIM_TIME + expon(MEAN_BREAK) for _ in range(NUM_MACHINES)] + [np.inf] * NUM_MACHINES + [800.0]

    NUM_EVENTS = len(TIME_NEXT_EVENT)
    TIME_LAST_EVENT = 0.0

    # state vars
    NUM_IN_Q = 0
    MACHINE_STATUS = [0] * NUM_MACHINES # 0 = working; 1 = broken
    TECHNICIAN_STATUS = [-1] * NUM_TECHNICIANS # index of machine being repaired, or -1

    # stats
    NUM_CUSTS_DELAYED = 0
    TOTAL_OF_DELAYS = 0.0
    AREA_NUM_IN_Q = 0.0
    TOTAL_TECH_COST = 0.0
    TOTAL_BREAKDOWN_COST = 0.0


# def event():
#     # 1. state changes are made
#     # 2. future events are scheduled
#     # 3. event is removed from event list

def machine_breaks(m: int):
    """m is 0-2, index of machine"""
    # state changes are made
    global MACHINE_STATUS, SIM_TIME
    MACHINE_STATUS[m] = 1 # machine broken

    # schedule future events:
    # if queue is empty and a technician is available, start service immediately
    global NUM_IN_Q, TECHNICIAN_STATUS, TIME_NEXT_EVENT
    available_techs = [idx for idx, val in enumerate(TECHNICIAN_STATUS) if val == -1]
    if NUM_IN_Q == 0 and available_techs:
        # select a random technician and start work
        tech = np.random.choice(a=available_techs)
        TECHNICIAN_STATUS[tech] = m # start work on this machine
        # record the delay
        global TOTAL_OF_DELAYS, NUM_CUSTS_DELAYED
        NUM_CUSTS_DELAYED += 1
        TOTAL_OF_DELAYS += 0.0

        # schedule service completion
        global MEAN_REPAIR
        repair_completion = SIM_TIME + expon(MEAN_REPAIR)
        event_idx = m + len(MACHINE_STATUS)
        TIME_NEXT_EVENT[event_idx] = repair_completion

    # if queue is nonempty, join the queue
    else:
        NUM_IN_Q += 1
        global Q_TIME_ARRIVAL, Q_MACHINE
        Q_TIME_ARRIVAL.append(SIM_TIME)
        Q_MACHINE.append(m)

    # remove machine break event from event list
    event_idx = m
    TIME_NEXT_EVENT[m] = np.inf


def machine_service_ends(m: int):
    """m is 0-2, index of machine
        Find technician working on this machine. If there is a queue, start
        work immediately; otherwise, become idle.
    """
    # change state variables
    global SIM_TIME, MACHINE_STATUS, TIME_NEXT_EVENT
    MACHINE_STATUS[m] = 0 # machine is fixed

    global TECHNICIAN_STATUS
    tech_idx = np.argwhere(np.array(TECHNICIAN_STATUS) == m)[0][0]

    global NUM_IN_Q
    if NUM_IN_Q == 0:
        # no machines are waiting, so technician becomes idle
        TECHNICIAN_STATUS[tech_idx] = -1
    else:
        # start repair on next machine in queue
        NUM_IN_Q -= 1
        global Q_MACHINE, Q_TIME_ARRIVAL
        m_idx = Q_MACHINE[0]
        delay = SIM_TIME - Q_TIME_ARRIVAL[0]

        global TOTAL_OF_DELAYS, NUM_CUSTS_DELAYED
        NUM_CUSTS_DELAYED += 1
        TOTAL_OF_DELAYS += delay

        TECHNICIAN_STATUS[tech_idx] = m_idx
        # schedule repair completion
        global MEAN_REPAIR
        repair_done = SIM_TIME + expon(MEAN_REPAIR)
        event_idx = m_idx + len(MACHINE_STATUS)
        TIME_NEXT_EVENT[event_idx] = repair_done

        # shift queue
        Q_MACHINE = Q_MACHINE[1:]
        Q_TIME_ARRIVAL = Q_TIME_ARRIVAL[1:]


    # schedule future events
    # next machine breakage
    global MEAN_BREAK
    next_break = SIM_TIME + expon(MEAN_BREAK)
    TIME_NEXT_EVENT[m] = next_break

    # remove service end event from list
    event_idx = len(MACHINE_STATUS) + m
    TIME_NEXT_EVENT[event_idx] = np.inf

def timing():
    """Compare the timings of the next of each possible event type to find
    the next event. Set NEXT_EVENT_TYPE accordingly, and advance the sim
    clock to the time of occurrent of this event.
    """
    global NEXT_EVENT_TYPE, NUM_EVENTS, TIME_NEXT_EVENT, SIM_TIME

    min_time_next_event = np.inf

    for i in range(NUM_EVENTS):
        if TIME_NEXT_EVENT[i] < min_time_next_event:
            min_time_next_event = TIME_NEXT_EVENT[i]
            NEXT_EVENT_TYPE = i

    SIM_TIME = TIME_NEXT_EVENT[NEXT_EVENT_TYPE]

def update_time_avg_stats():
    """Prior to processing each event, this function updates the areas tracked
    for the calculation of continuous-time statistics."""
    global SIM_TIME, TIME_LAST_EVENT
    time_since_last_event = SIM_TIME - TIME_LAST_EVENT
    TIME_LAST_EVENT = SIM_TIME # current time

    # update area under num-in-queue function
    global AREA_NUM_IN_Q, NUM_IN_Q
    AREA_NUM_IN_Q += NUM_IN_Q * time_since_last_event

    # update technician cost
    global TOTAL_TECH_COST, NUM_TECHNICIANS
    TOTAL_TECH_COST += 10.00 * NUM_TECHNICIANS * time_since_last_event

    # update breakdown cost
    global TOTAL_BREAKDOWN_COST, MACHINE_STATUS
    num_machines_broken = sum(MACHINE_STATUS)
    TOTAL_BREAKDOWN_COST += 50.00 * num_machines_broken * time_since_last_event



def report():
    """Log statistics"""
    global TOTAL_OF_DELAYS, NUM_CUSTS_DELAYED, AREA_NUM_IN_Q, SIM_TIME, AREA_SERVER_STATUS, TOTAL_TECH_COST, TOTAL_BREAKDOWN_COST, NUM_TECHNICIANS
    # print(f"\n\nAverage delay in queue     {TOTAL_OF_DELAYS / NUM_CUSTS_DELAYED:11.3f} minutes\n\n")
    # print(f"Average number in queue     {AREA_NUM_IN_Q / SIM_TIME:10.3f}\n\n")
    # print(f"Time simulation ended       {SIM_TIME:12.3f} minutes\n\n")
    print(f"\n\nPolicy: {NUM_TECHNICIANS} technicians")
    print(f"Total technician cost        ${TOTAL_TECH_COST:,.2f}")
    print(f"Total breakdown cost         ${TOTAL_BREAKDOWN_COST:,.2f}")
    print(f"Total cost                   ${TOTAL_TECH_COST + TOTAL_BREAKDOWN_COST:,.2f}")

def main():
    global NUM_TECHNICIANS

    for nt in range(1,6):
        NUM_TECHNICIANS = nt
        initialize()
        while True:
            timing()

            update_time_avg_stats()
            
            if NEXT_EVENT_TYPE in list(range(NUM_MACHINES)):
                machine_breaks(m=NEXT_EVENT_TYPE)
            elif NEXT_EVENT_TYPE in range(NUM_MACHINES, 2*NUM_MACHINES):
                machine_service_ends(m=NEXT_EVENT_TYPE - len(MACHINE_STATUS))
            else:
                report()
                break

main()



Policy: 1 technicians
Total technician cost        $8,000.00
Total breakdown cost         $80,400.12
Total cost                   $88,400.12


Policy: 2 technicians
Total technician cost        $16,000.00
Total breakdown cost         $42,991.92
Total cost                   $58,991.92


Policy: 3 technicians
Total technician cost        $24,000.00
Total breakdown cost         $43,030.51
Total cost                   $67,030.51


Policy: 4 technicians
Total technician cost        $32,000.00
Total breakdown cost         $38,259.78
Total cost                   $70,259.78


Policy: 5 technicians
Total technician cost        $40,000.00
Total breakdown cost         $41,345.75
Total cost                   $81,345.75
